In [1]:
# 모듈 불러오기
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# FASHION MNIST 데이터셋 불러오기
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data()

# 데이터셋 정규화
train_X = train_X / 255.0
test_X = test_X / 255.0

# 데이터셋 shape 확인
print(train_X.shape, test_X.shape)

# 데이터셋에 차원(채널) 추가
train_X = train_X.reshape(60000, 28, 28, 1) # 2차원(28*28)에서 3차원(28*28*1)으로 변환
test_X = test_X.reshape(10000, 28, 28, 1) # 2차원(28*28)에서 3차원(28*28*1)으로 변환

# reshape 이후 shape 확인
print(train_X.shape, test_X.shape)

In [3]:
# 데이터 확인
plt.figure(figsize=(10, 10))  #전체 그래프 사이즈를 width=10, height=10 으로 지정
for c in range(16):
    plt.subplot(4, 4, c+1)  #4행 4열로 지정한 grid 에서 c+1 번째의 칸에 그래프 그림. 
    plt.imshow(train_X[c].reshape(28,28), cmap="gray")    
plt.show()

#train 데이터의 첫번째 ~ 16번째 까지의 라벨을 프린트
print(train_Y[:16])

In [4]:
# 컨볼루션 신경망 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=16),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=32),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# 계층, 차원, 파라미터 수 요약 확인
model.summary()

In [ ]:
# 학습실행 : 학습데이터에서 검증 데이터 25% 분리, epochs 25회 후 손실 및 정확도 출력
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

In [ ]:
# 학습결과 확인 : train loss, validation loss 그래프 출력
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], "b-", label="loss")
plt.plot(history.history["val_loss"], "r--", label="val_loss")
plt.xlabel("Epoch")
plt.legend()

# 학습결과 확인 : train accuracy, validation accuracy 그래프 출력
plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], "g-", label="accuracy")
plt.plot(history.history["val_accuracy"], "k--", label="val_accuracy")
plt.xlabel("Epoch")
plt.ylim(0.7, 1)
plt.legend()
plt.show()

# 테스트 데이터에 대한 손실 및 정확도 출력
model.evaluate(test_X, test_Y, verbose=0)

In [ ]:
# 에포크가 증가할 수록 validation loss가 증가하고, validation accuracy는 감소하고 있어 과적합 상태임
# test data에 대한 loss와 accuracy가 train data에 비해 낮음
# 성능 향상을 위해 모델 재설계 필요

# 과적합 해소를 위해 pooling layer, drop out 추가
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=32),
    tf.keras.layers.MaxPool2D(strides=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64),
    tf.keras.layers.MaxPool2D(strides=(2, 2)),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

In [ ]:
# 학습실행 : 학습데이터에서 검증 데이터 25% 분리, epochs 25회 후 손실 및 정확도 출력
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

In [ ]:
# 학습결과 확인 : train loss, validation loss 그래프 출력
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], "b-", label="loss")
plt.plot(history.history["val_loss"], "r--", label="val_loss")
plt.xlabel("Epoch")
plt.legend()

# 학습결과 확인 : train accuracy, validation accuracy 그래프 출력
plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], "g-", label="accuracy")
plt.plot(history.history["val_accuracy"], "k--", label="val_accuracy")
plt.xlabel("Epoch")
plt.ylim(0.7, 1)
plt.legend()
plt.show()

# 테스트 데이터에 대한 손실 및 정확도 출력
model.evaluate(test_X, test_Y, verbose=0)

In [ ]:
# VGGnet 유형으로 모델 정의  제로패딩, activation relu 사용
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(28, 28,1), kernel_size=(3, 3), filters=32, padding="same", activation="relu"),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding="same", activation="relu"),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=256, padding= "same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation="relu"),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
])
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])
model.summary()

In [ ]:
# 학습실행 : 학습데이터에서 검증 데이터 25% 분리, epochs 25회 후 손실 및 정확도 출력
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

In [ ]:
# 학습결과 확인 : train loss, validation loss 그래프 출력
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"], "b-", label="loss")
plt.plot(history.history["val_loss"], "r--", label="val_loss")
plt.xlabel("Epoch")
plt.legend()

# 학습결과 확인 : train accuracy, validation accuracy 그래프 출력
plt.subplot(1, 2, 2)
plt.plot(history.history["accuracy"], "g-", label="accuracy")
plt.plot(history.history["val_accuracy"], "k--", label="val_accuracy")
plt.xlabel("Epoch")
plt.ylim(0.7, 1)
plt.legend()
plt.show()

# 테스트 데이터에 대한 손실 및 정확도 출력
model.evaluate(test_X, test_Y, verbose=0)